In [20]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from datetime import datetime, timedelta
import ccxt
import ta
from scipy import stats
from sklearn import preprocessing

In [34]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

def execute_query(connection, query, varlist=None):
    cursor = connection.cursor(buffered=True)
    try:
        if varlist is None:
            cursor.execute(query)
        else:
            cursor.execute(query, varlist)
        connection.commit()
        print("Query successful", end='\r')
        return cursor.fetchall()
    except Error as err:
        print(f"Error: '{err}'")

def executemany_query(connection, query, varlist=None):
    cursor = connection.cursor()
    try:
        if varlist is None:
            cursor.executemany(query)
        else:
            cursor.executemany(query, varlist)
        connection.commit()
        print("Query successful", end='\r')
    except Error as err:
        print(f"Error: '{err}'")

In [35]:
pw = '27111998'
db = 'crypto_db'
connection = create_db_connection("localhost", "root", pw, db)

MySQL Database connection successful


In [36]:
exchange_id = 'tokocrypto'
exchange_class = getattr(ccxt, exchange_id)
exchange = exchange_class({
    'apiKey': 'YOUR_API_KEY',
    'secret': 'YOUR_SECRET',
})
markets = exchange.load_markets()
market_list = list(markets.keys())
considered_market = []
for i in market_list:
    if '/USDT' in i or '/BTC' in i:
        considered_market.append(i)

print(len(considered_market))
print(considered_market)

604
['1INCH/BTC', '1INCH/USDT', 'AAVE/USDT', 'ACA/BTC', 'ACA/USDT', 'ACE/BTC', 'ACE/USDT', 'ACH/BTC', 'ACH/USDT', 'ACM/USDT', 'ADA/BTC', 'ADA/USDT', 'ADX/BTC', 'ADX/USDT', 'AERGO/BTC', 'AERGO/USDT', 'AGIX/BTC', 'AGIX/USDT', 'AGLD/BTC', 'AGLD/USDT', 'AI/BTC', 'AI/USDT', 'AKRO/USDT', 'ALCX/BTC', 'ALCX/USDT', 'ALGO/BTC', 'ALGO/USDT', 'ALICE/BTC', 'ALICE/USDT', 'ALPACA/BTC', 'ALPACA/USDT', 'ALPHA/BTC', 'ALPHA/USDT', 'ALPINE/BTC', 'ALPINE/USDT', 'ALT/BTC', 'ALT/USDT', 'AMP/USDT', 'ANKR/BTC', 'ANKR/USDT', 'APE/BTC', 'APE/USDT', 'API3/BTC', 'API3/USDT', 'APT/BTC', 'APT/USDT', 'ARB/BTC', 'ARB/USDT', 'ARKM/BTC', 'ARKM/USDT', 'ARPA/BTC', 'ARPA/USDT', 'AR/BTC', 'AR/USDT', 'ASR/USDT', 'ASTR/USDT', 'AST/BTC', 'ATA/BTC', 'ATA/USDT', 'ATM/USDT', 'ATOM/BTC', 'ATOM/USDT', 'AUCTION/BTC', 'AUCTION/USDT', 'AUDIO/BTC', 'AUDIO/USDT', 'AVAX/BTC', 'AVAX/USDT', 'AVA/BTC', 'AVA/USDT', 'AXS/BTC', 'AXS/USDT', 'BADGER/BTC', 'BADGER/USDT', 'BAL/BTC', 'BAL/USDT', 'BAND/BTC', 'BAND/USDT', 'BAR/USDT', 'BAT/BTC', 'BAT/

In [37]:
shortlisted_market = []
shortlisted_market_dict = {}
for i in range(len(considered_market)):
    try:
        market = considered_market[i]
        market = market.replace('/', '')
        print(market, end='\r')
        query = 'SELECT * FROM %s ORDER BY timestamp ASC' % market
        res = execute_query(connection,query)
        df = pd.DataFrame(res,columns=['timestamp', 'open','high', 'low','close','volume'])
        df = df.drop_duplicates()
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

        is_adi_increase = False
        adi = ta.volume.AccDistIndexIndicator(high=df['high'],low=df['low'],close=df['close'],volume=df['volume']).acc_dist_index()
        x = df.iloc[(-24*7):].index.values/24
        y = adi.iloc[(-24*7):]
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
        if slope > 0:
            is_adi_increase = True
        is_aroon = False
        day_df = (df.resample('D', on='timestamp').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last','volume': 'mean'}))
        day_df['timestamp'] = day_df.index
        aroon = ta.trend.AroonIndicator(low=day_df['low'],high=day_df['high'])
        aroon_up = aroon.aroon_up()
        aroon_down = aroon.aroon_down()
        aroon_diff = aroon.aroon_indicator()

        if aroon_diff[-1] > 0 and aroon_diff[-1] < 10:
            is_aroon = True
        elif np.mean(aroon_up[-3:]) > 85:
            is_aroon = True

        is_macd = False
        macd = ta.trend.MACD(close=day_df['close'])
        macd_diff = macd.macd_diff()
        scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
        macd_diff = scaler.fit_transform(macd_diff.values.reshape(-1,1))
        macd_line = macd.macd()
        macd_signal = macd.macd_signal()

        if abs(macd_diff[-1]) < 0.015:
            is_macd = True
        elif (macd_line[-1] - macd_signal[-1]) >= 0 and (macd_line[-1] - macd_signal[-1])/macd_line[-1] <= 0.10 :
            is_macd = True
        is_stochastic = False
        stoch_oscillator = ta.momentum.StochasticOscillator(high=day_df['high'], low=day_df['low'], close=day_df['close'])
        stochastic = stoch_oscillator.stoch()
        stochastic_signal = stoch_oscillator.stoch_signal()

        if stochastic.values[-1] < 20 and stochastic_signal[-1] < 20:
            is_stochastic = True

        if sum([is_adi_increase, is_aroon, is_macd, is_stochastic]) >= 3:
            shortlisted_market.append(market)
            shortlisted_market_dict[market] = {'is_adi_increase': is_adi_increase,
                                               'is_aroon': is_aroon,
                                               'is_macd': is_macd,
                                               'is_stochastic': is_stochastic}
    except Exception as e:
        print(e)
        continue
    

e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:479: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:479: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


Inputs must not be empty.


e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:479: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:479: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:479: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:479: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:479: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


Error: '1146 (42S02): Table 'crypto_db.pixelbtc' doesn't exist'
Inputs must not be empty.
Error: '1146 (42S02): Table 'crypto_db.pixelusdt' doesn't exist'
Inputs must not be empty.


e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:479: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:479: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:479: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:479: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
e:\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


Inputs must not be empty.


In [38]:
shortlisted_market

['APTUSDT',
 'ARBTC',
 'CTSIUSDT',
 'HARDUSDT',
 'ICPUSDT',
 'ILVUSDT',
 'LPTUSDT',
 'LSKUSDT',
 'NULSBTC',
 'QUICKBTC',
 'REEFUSDT',
 'SANDUSDT',
 'STGBTC',
 'SUIUSDT',
 'TRBBTC',
 'UNFIUSDT',
 'XLMUSDT']

In [42]:
shortlisted_market_dict['TRBBTC']

{'is_adi_increase': True,
 'is_aroon': False,
 'is_macd': True,
 'is_stochastic': True}

In [8]:
is_aroon = False
day_df = (df.resample('D', on='timestamp').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last','volume': 'mean'}))
day_df['timestamp'] = day_df.index
aroon = ta.trend.AroonIndicator(low=day_df['low'],high=day_df['high'])
aroon_up = aroon.aroon_up()
aroon_down = aroon.aroon_down()
aroon_diff = aroon.aroon_indicator()

if aroon_diff[-1] > 0 and aroon_diff[-1] < 10:
    is_aroon = True
elif np.mean(aroon_up[-3:]) > 85:
    is_aroon = True

In [18]:
stochastic

timestamp
2018-04-17          NaN
2018-04-18          NaN
2018-04-19          NaN
2018-04-20          NaN
2018-04-21          NaN
                ...    
2024-02-13    77.551020
2024-02-14    94.122966
2024-02-15    97.212294
2024-02-16    89.101203
2024-02-17    81.457891
Freq: D, Name: stoch_k, Length: 2133, dtype: float64

In [ ]:


btc_correlation = 0

In [5]:
a = "\u0030" #unicode for 0
b = "\u00B2" #unicode for &sup2;
c = "10km2"
d = "-1"
e = "1.5"

print(a.isnumeric())
print(b.isnumeric())
print(c.isnumeric())
print(d.isnumeric())
print(e.isnumeric())

True
True
False
False
False


In [9]:
float('-291.302et')+10

ValueError: could not convert string to float: '-291.302et'

In [15]:
import numpy as np
a = np.array([-2,3,4,-2])
b = np.array
np.where(a > 0, a, 0)

array([0, 3, 4, 0])

In [27]:
import re

x = 'skajcbakjsbc>=PetalWidthCm'
a = re.findall('(.+?)[><=!]+',x)[0]
# b = re.findall('[><=!]+(.+?)',x)[0]
b = re.findall('[><=!]+(.+)', x)[0]


print(a,b)

skajcbakjsbc PetalWidthCm
